### Compute Reduction in Features

In [25]:
MONGO_COLLECTION = "CB_RE-RANKER_HYPER_PARAM_VD"

In [26]:
# for r in rows[-17:]:
#     print(str(r["asof"]))

In [27]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [28]:
df = pd.DataFrame(results)
df.tail(3)

,C,accuracy,asof,best_max_parses,best_max_upd,best_min_prob,best_top_n,data_points,early_stopping_iters,extractors,...,max_epochs,max_update_items,min_feat_freq,num_codes,num_feats_MEAN,num_feats_per_fold,num_tags,pa_type,precision,recall
614,0.01,0.983431,2019-06-17 05:00:51.663000,300,NaN,0.0,3,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,2745.4,"[2706, 2596, 2682, 2861, 2882]",91,1,0.799739,0.671898
615,0.01,0.983577,2019-06-17 05:08:00.875000,300,NaN,0.0,3,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,2745.4,"[2706, 2596, 2682, 2861, 2882]",91,1,0.792929,0.687591
616,0.01,0.982530,2019-06-17 05:16:02.667000,300,NaN,0.0,3,82082.0,3,"[CREL_, Prob-, CChainStats-]",...,3,1,1,2740.0,2745.4,"[2706, 2596, 2682, 2861, 2882]",91,1,0.758103,0.700000


In [29]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
cols = ("f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", "best_min_prob", 
    #"extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type")
df[list(cols)].head(10)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,best_min_prob,initial_weight,loss_type,pa_type
195,0.746779,0.789110,0.708759,2019-06-14 01:50:35.450000,2,0.0100,300,1,0.0,0.01,ml,1
319,0.745815,0.788767,0.707299,2019-06-15 00:06:57.168000,2,0.0025,300,10,0.0,0.01,pb,1
39,0.745737,0.784994,0.710219,2019-06-13 19:35:28.095000,2,0.0025,300,2,0.0,0.01,ml,1
163,0.745731,0.776680,0.717153,2019-06-14 00:39:56.654000,2,0.0025,300,1,0.0,0.01,ml,1
175,0.745638,0.795285,0.701825,2019-06-14 01:06:17.901000,2,0.0025,300,10,0.0,0.01,ml,1
171,0.745462,0.791632,0.704380,2019-06-14 00:56:54.486000,2,0.0025,300,5,0.0,0.01,ml,1
170,0.745402,0.793814,0.702555,2019-06-14 00:54:34.912000,2,0.0025,300,5,0.0,0.00,ml,1
151,0.745399,0.785137,0.709489,2019-06-14 00:08:20.922000,2,0.0005,300,2,0.0,0.01,ml,2
46,0.745342,0.796020,0.700730,2019-06-13 19:50:52.563000,2,0.0025,300,10,0.0,0.00,ml,1
179,0.745318,0.791308,0.704380,2019-06-14 01:15:14.959000,2,0.0025,300,1,0.0,0.01,ml,2


In [30]:
len(df[df.best_top_n == 3])

53